In [78]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [79]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [80]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [81]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [82]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [83]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [84]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [85]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [86]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [87]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [88]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [89]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [90]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [91]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [92]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  6385,  1149, 11238], shape=(11314,))

In [93]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [94]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [95]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [96]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [97]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [98]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.



**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [99]:
# hago los mismo pasos para el vectorizador
tfidfvect = TfidfVectorizer()
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# entreno y ajusto el vectorizador de entrenamiento
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [100]:
def print_similar_documents(idx):
  cossim = cosine_similarity(X_train[idx], X_train)[0]
  
  # podemos ver los valores de similaridad ordenados de mayor a menos, nos quedamos con los 5 mejores
  mostsim = np.argsort(cossim)[::-1][1:6]
  
  print(f"El documento '{idx}' es de la clase '{newsgroups_train.target_names[y_train[idx]]}'")
  print(f"Su contenido es: \t{newsgroups_train.data[idx]}")
  print(f"Las 5 clases más similares son:")
  for i in mostsim:
    print(newsgroups_train.target_names[y_train[i]])


In [101]:
# Elegí 5 indices de documentos al azar
my_random_documents_idx = [14, 8, 100, 103, 10000]

In [102]:
print_similar_documents(my_random_documents_idx[0])

El documento '14' es de la clase 'misc.forsale'
Su contenido es: 	Reduced Prices! 
I have a list of things forsale on behalf of my brother, who's moving (moved
already)

								Offer:
1) Black and Decker Duster Plus (Portable Hand Vaccum)	
 	purchased for $32, 					  $12

2) SR-1000 Dual Cassette Portable Player, AM/FM
5-Band graphics Equalizer, high speed dubing, Duo 
Tape.Tape deck A, seems to have lost treble sound. 
But, I bet  it's fixable.
	purchased for $80					  $25

3)Monolux Zoom MicroScope, up to 1200X magnification
Made in Japan, includes case and accessories
	purchased for $50					  $20

4)Sunbeam 1400 Hair Dryer, the dryer you put your 
head under/into. You know, the ones you see in the salons.
(Don't ask me why my bro had it)
	purchased for $60				          $24

5)Everylast Speed Bag, all leather. Brand new, never 
used								  $10

6)Osterizer Pusle Matic Blender, with 10 speeds 
and a cookbook, 5 years old					  $10
	purchased for $50

8)Binolux Binoculars . 7x35, ex

Explicación:
- Está perfecto que la mejor similitud la tiene con "misc.forsale".
- Con "rec.motorcyles" tal vez porque tiene las palabras: "hand", "accessories", "Iron", "speed".
- Con "comp.os.ms-windows.misc" tal vez porque tiene las palabras: "accessories", "Dual Cassette Portable", "case", "e-mail"
- Con "soc.religion.christian" tal vez porque tiene las palabras: "brother", "high". No tiene mucho sentido, pero es el quinto con mayor similitud.

In [103]:
print_similar_documents(my_random_documents_idx[1])

El documento '8' es de la clase 'comp.os.ms-windows.misc'
Su contenido es: 	I have win 3.0 and downloaded several icons and BMP's but I can't figure out
how to change the "wallpaper" or use the icons.  Any help would be appreciated.


Thanx,

-Brando
Las 5 clases más similares son:
comp.windows.x
comp.os.ms-windows.misc
comp.windows.x
comp.os.ms-windows.misc
comp.graphics


Explicación:
- El texto trata un sistema operativo lo cual es correcto que esté relacionado con "comp.windows.x", que es una versión de windows, y aparece en el primer y tercer lugar. También aparece "comp.os.ms-windows.misc" que está relacionado con windows. Lo último que aparece es "comp.graphics" que puede ser que sea por las palabras "wallpaper", "icons", "win".

In [104]:
print_similar_documents(my_random_documents_idx[2])

El documento '100' es de la clase 'misc.forsale'
Su contenido es: 	1.  Software publishing SuperBase 4 windows v.1.3           --->$80

2.  OCR System ReadRight v.3.1 for Windows                  --->$65

3.  OCR System ReadRight  v.2.01 for DOS                    --->$65

4.  Unregistered Zortech 32 bit C++ Compiler v.3.1          --->$ 250
     with Multiscope windows Debugger,
     WhiteWater Resource Toolkit, Library Source Code

5.  Glockenspiel/ImageSoft Commonview 2 Windows
     Applications Framework for Borland C++                 --->$70

6.  Spontaneous Assembly Library With Source Code           --->$50

7.  Microsoft Macro Assembly 6.0                            --->$50

8.  Microsoft Windows v.3.1 SDK Documentation               --->$125

9.  Microsoft FoxPro V.2.0                                  --->$75

10.  WordPerfect 5.0 Developer's Toolkit                    --->$20

11.  Kedwell Software DataBoss v.3.5 C Code Generator       --->$100

12.  Kedwell InstallBoss v.2.

Explicación:
- La clase de este documento no coincide con las 5 clases mas similares. El algoritmo lo asocia con computación, windows y hardware de ibm. Tiene sentido por el hecho que en la oferta se detallan items de computación.

In [105]:
print_similar_documents(my_random_documents_idx[3])

El documento '103' es de la clase 'comp.sys.mac.hardware'
Su contenido es: 	
Tests suck! Post a real message!
:^)

Las 5 clases más similares son:
rec.sport.baseball
sci.med
rec.sport.hockey
sci.space
rec.autos


Explicación:

- El documento es corto y el mensaje es bastante genérico lo cual no lo asocia a su clase. Lo asocia a deportes como baseball y hockey tal vez por mencionar las palabras: "tests", "suck". Y el resto de las clase tal vez por las palabras "tests" y "real" que pueden tener relacion a pruebas en ciencia y autos.

In [106]:
print_similar_documents(my_random_documents_idx[4])

El documento '10000' es de la clase 'rec.autos'
Su contenido es: 	I have to disagree with this.  I have a 92 Z28 with a 350 and a 4-speed auto
w/ overdrive, and it is really better that way.  Chevy autos are reknowned
for their long life and ability to handle copious amount of power.  I live 
in the Dallas area, and a manual would be much harder to drive in the traffic 
here.  Now if I still lived out in the sticks like I used to, a manual would be
more fun.  

Safety-wise, an auto is less distracting...I would hate to have to be    
shifting gears while I was trying to ease into traffic in the freeways here.
Performance-wise, I can hold my own against any stock 5.0 Mustang or 5.0
Camaro w/ a five speed.  

All of this IMHO... :)




Las 5 clases más similares son:
rec.autos
rec.autos
talk.politics.guns
rec.autos
sci.electronics


Explicación:
- El documento es de clase "rec.autos" y lo vemos en 3 acosaciones. Tambien lo relaciona mucho con "talk.politics.guns" tal vez por las palabras: "safety-wise", "against", "disagree", "life", "power", "hate". Tambien lo relaciona con electronics lo cual tiene sentido porque menciona: "speed", "manual", "4-speed".

### **2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

Clasificación por prototipos (similaridad coseno a centroides de clase)
Para cada documento de test se computa similaridad coseno contra un prototipo por clase (centroide TF-IDF de los documentos de entrenamiento de esa clase) y se asigna la clase con mayor similaridad. Se reporta F1 macro.


In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix, vstack
import numpy as np

# Vectorización
tfidfvect = TfidfVectorizer()
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

# Construcción de centroides por clase (promedio TF-IDF por clase, luego normalizado L2)
classes = np.unique(y_train)
prototypes = []
for c in classes:
    Xc = X_train[y_train == c]
    proto = Xc.mean(axis=0)  # 1 x n_features (numpy matrix)
    # Convierte el vector proto (el promedio de las muestras de una clase) en una matriz dispersa (sparse matrix) en formato CSR (Compressed Sparse Row).
    # El formato CSR guarda sólo los valores distintos de cero y sus posiciones.
    proto = csr_matrix(proto)
    # Normaliza para que la similitud del coseno entre vectores sea equivalente al producto punto.
    proto = normalize(proto, norm='l2')
    prototypes.append(proto)

# Cada fila representa el prototipo de una clase, shape: (n_classes, n_features)
class_prototypes = vstack(prototypes)  

# Similaridad coseno contra cada centroide y asignación de la clase más similar
S = cosine_similarity(X_test, class_prototypes)  # (n_test, n_classes)
y_pred_proto = classes[np.argmax(S, axis=1)]

f1_macro = f1_score(y_test, y_pred_proto, average='macro')
print(f"F1 macro (prototipos por centroides coseno): {f1_macro:.4f}")


F1 macro (prototipos por centroides coseno): 0.6092


In [108]:
# Muestro 5 ejemplos de clasificación: texto, verdadera, predicha, correcto o no
for i in range(5):
    idx = i
    texto = newsgroups_test.data[idx]
    verdadera = newsgroups_test.target_names[y_test[idx]]
    predicha = newsgroups_test.target_names[y_pred_proto[idx]]
    correcto = (y_test[idx] == y_pred_proto[idx])
    print(f"Ejemplo {i+1}:")
    print(f"Texto: {texto[:300].replace('\n', ' ')}...")
    print(f"Clase verdadera: {verdadera}")
    print(f"Clase predicha: {predicha}")
    print(f"Correcto: {correcto}")
    print("-"*60)


Ejemplo 1:
Texto: I am a little confused on all of the models of the 88-89 bonnevilles. I have heard of the LE SE LSE SSE SSEI. Could someone tell me the differences are far as features or performance. I am also curious to know what the book value is for prefereably the 89 model. And how much less than book value can...
Clase verdadera: rec.autos
Clase predicha: rec.autos
Correcto: True
------------------------------------------------------------
Ejemplo 2:
Texto: I'm not familiar at all with the format of these "X-Face:" thingies, but after seeing them in some folks' headers, I've *got* to *see* them (and maybe make one of my own)!  I've got "dpg-view" on my Linux box (which displays "uncompressed X-Faces") and I've managed to compile [un]compface too... but...
Clase verdadera: comp.windows.x
Clase predicha: comp.graphics
Correcto: False
------------------------------------------------------------
Ejemplo 3:
Texto:  In a word, yes. ...
Clase verdadera: alt.atheism
Clase predicha: talk

El modelo basado en centroides de TF-IDF y similitud coseno obtuvo un f1_score macro cercano a 0.6092 en el conjunto de test.
Esto indica que si bien es mejor que el azar, el clasificador no logra distinguir bien entre las distintas categorías,
por lo que no es una clasificación particularmente buena, más bien es básico. Probablemente sea necesario utilizar modelos más sofisticados
o ajustar mejor los parámetros para mejorar el desempeño.


### **3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

In [109]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

best_f1 = 0
best_desc = ""
best_model = None

alphas = [0.1, 0.5, 0.75, 1.0]
max_dfs = [0.6, 0.7, 0.75, 0.8]
min_dfs = [1, 2, 3, 5]

for vec_name in ["CountVectorizer", "TfidfVectorizer"]:
    for max_df in max_dfs:
        for min_df in min_dfs:
            if vec_name == "CountVectorizer":
                vectorizer = CountVectorizer(
                    stop_words="english",
                    max_df=max_df,
                    min_df=min_df,
                )
            else:
                vectorizer = TfidfVectorizer(
                    stop_words="english",
                    max_df=max_df,
                    min_df=min_df,
                    norm="l2",
                    use_idf=True,
                )
            X_train_vec = vectorizer.fit_transform(newsgroups_train.data)
            X_test_vec = vectorizer.transform(newsgroups_test.data)
            for model_name in ["MultinomialNB", "ComplementNB"]:
                for alpha in alphas:
                    if model_name == "MultinomialNB":
                        model = MultinomialNB(alpha=alpha)
                    else:
                        model = ComplementNB(alpha=alpha)
                    model.fit(X_train_vec, newsgroups_train.target)
                    y_pred = model.predict(X_test_vec)
                    f1 = f1_score(newsgroups_test.target, y_pred, average="macro")
                    desc = (
                        f"{model_name} con {vec_name} (max_df={max_df}, min_df={min_df}, alpha={alpha}): "
                        f"F1 macro = {f1:.4f}"
                    )
                    print(desc)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_desc = desc
                        best_model = (vectorizer, model)

print("\nMejor configuración:")
print(best_desc)

MultinomialNB con CountVectorizer (max_df=0.6, min_df=1, alpha=0.1): F1 macro = 0.6320
MultinomialNB con CountVectorizer (max_df=0.6, min_df=1, alpha=0.5): F1 macro = 0.6182
MultinomialNB con CountVectorizer (max_df=0.6, min_df=1, alpha=0.75): F1 macro = 0.6025
MultinomialNB con CountVectorizer (max_df=0.6, min_df=1, alpha=1.0): F1 macro = 0.5968
ComplementNB con CountVectorizer (max_df=0.6, min_df=1, alpha=0.1): F1 macro = 0.6404
ComplementNB con CountVectorizer (max_df=0.6, min_df=1, alpha=0.5): F1 macro = 0.6454
ComplementNB con CountVectorizer (max_df=0.6, min_df=1, alpha=0.75): F1 macro = 0.6455
ComplementNB con CountVectorizer (max_df=0.6, min_df=1, alpha=1.0): F1 macro = 0.6433
MultinomialNB con CountVectorizer (max_df=0.6, min_df=2, alpha=0.1): F1 macro = 0.6284
MultinomialNB con CountVectorizer (max_df=0.6, min_df=2, alpha=0.5): F1 macro = 0.6283
MultinomialNB con CountVectorizer (max_df=0.6, min_df=2, alpha=0.75): F1 macro = 0.6230
MultinomialNB con CountVectorizer (max_df=0.

Mejor configuración:
ComplementNB con TfidfVectorizer (max_df=0.6, min_df=1, alpha=0.5): F1 macro = 0.6978

La combinación de ComplementNB y TfidfVectorizer resultó ser la más efectiva para el problema planteado pero con un F1 macro bajo, pero mejor el clasificador por prototipos mostrando la importancia de ajustar tanto el modelo como la vectorización de textos para mejorar el desempeño en clasificación de noticias.

### **4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

In [110]:
tfidfvect = TfidfVectorizer()
newsgroups_train = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# es muy útil tener el diccionario opuesto que va de índices a términos
# idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

print(X_train.shape)
X_train_T = X_train.T
print(X_train_T.shape)


(18846, 134410)
(134410, 18846)


In [111]:
print(f"Hay {X_train_T.shape[0]} filas en X_train_T, coincide con la cantidad de términos {len(tfidfvect.vocabulary_)}")

Hay 134410 filas en X_train_T, coincide con la cantidad de términos 134410


In [112]:
idx1 = tfidfvect.vocabulary_['car']
print(idx1)
#cossim = cosine_similarity(X_train[idx], X_train)[0]
y_train = newsgroups_train.target

34680


In [113]:
def print_similar_terms(term):
  idx = tfidfvect.vocabulary_[term]

  X = X_train  # matriz TF-IDF de forma (n_docs, n_terms), esparsa

  # Similaridad coseno del término 'car' vs todos los términos (columnas)
  sims = cosine_similarity(X.T[idx, :], X.T).ravel()

  # Excluir el propio término
  sims[idx] = -1.0

  top_k = 5  # cantidad de términos a mostrar
  top_idx = np.argsort(sims)[::-1][:top_k]

  try:
      feature_names = tfidfvect.get_feature_names_out()
  except AttributeError:
      feature_names = tfidfvect.get_feature_names()

  top_terms = [(feature_names[i], float(sims[i])) for i in top_idx]
  print(f"Términos mas comunes con {term}:")
  print(top_terms)  # lista de (término, similitud)

In [114]:
print_similar_terms("car")

Términos mas comunes con car:
[('cars', 0.1930312685634614), ('owner', 0.156279214823919), ('engine', 0.1511692353481934), ('dealer', 0.1474968048065396), ('ford', 0.1444147192304934)]


Explicación:
- Tiene sentido que "car" esté relacionado con "cars" porque es el plural.
- "owner" porque puede ser en los textos diga "owner" del vehículo.
- "engine" porque es una parte de un auto.
- "dealer" podría ser quién maneja el auto.
- "ford" porque es una marca de autos.

In [115]:
print_similar_terms("baby")

Términos mas comunes con baby:
[('deformities', 0.30554572070991715), ('stillborn', 0.30554572070991715), ('circumsized', 0.2703558155243531), ('pregnant', 0.2655452252226731), ('deformed', 0.22483482354563564)]


Explicación:
- "deformities" y "deformed" porque se refieren a una forma anormal del cuerpo, tal vez los bebes.
- "stillborn" porque se refiere a una pérdida de embarazo.
- "circumsized" porque tal vez lo practican en bebes.
-"pregnant" porque la embaraza está muy relacionado a un bebe

In [116]:
print_similar_terms("phone")

Términos mas comunes con phone:
[('number', 0.21541910606127687), ('cellular', 0.19555401380902682), ('phones', 0.18329932547849462), ('dialtone', 0.1657260393361118), ('quicklinkii', 0.1657260393361118)]


Explicación:
- Los términos relacionados son: 'number', 'cellular', 'phones', 'dialtone', 'quicklinkii'. Todos están relacionados téfonos ya sea celulares o fijos.

In [117]:
print_similar_terms("email")

Términos mas comunes con email:
[('please', 0.27210690870136034), ('me', 0.19279202288212416), ('address', 0.16140030334930722), ('replies', 0.15974003788716126), ('thanks', 0.15885571865063774)]


Explicación:
Los términos son palabras que seguramente formaron parte de un correo electrónico, como: please, me, address, thanks, replies.

In [118]:
print_similar_terms("mother")

Términos mas comunes con mother:
[('constancy', 0.20501148099387673), ('irrigate', 0.20501148099387673), ('aa888', 0.20501148099387673), ('brood', 0.2048210828018481), ('father', 0.20285582312474135)]


Explicación:
- 'constancy' porque es una cualidad de las madres.
- 'irrigate' no se muy bien por qué estaría relacionado.
- 'aa888' porque es un tipo de placa madre
- 'brood' porque hay relación entre cria y madre.
- 'father' es su equivalente en masculino.